In [167]:
# !pip install google-cloud-storage
# !pip install subprocess
# !pip install google-drive
# !pip install google-api-python-client
# !pip install google-auth-httplib2 
# !pip install google-auth-oauthlib

In [168]:
import os
import warnings
import subprocess
import pandas as pd
from datetime import date
from google.cloud import bigquery as bq
from google.cloud import storage
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.api_core.exceptions import Forbidden, NotFound

In [169]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # show max columns

bq_key_path = "explore29-5a4f7581e39f.json"
bq_credentials = service_account.Credentials.from_service_account_file(bq_key_path)
bq_client = bq.Client(credentials=bq_credentials, project=bq_credentials.project_id)

bucket_key_path = "test_bucket_gdrive.json"
bucket_credentials = service_account.Credentials.from_service_account_file(bucket_key_path)
bucket_client = storage.Client(credentials=bucket_credentials, project=bucket_credentials.project_id)

In [170]:
def gen_file_name():
    return f'results_{str(date.today())}.csv'

In [171]:
def query_data():

	with open('test.sql', 'r') as file:
		query = ' '.join([line for line in file])

	file_name = gen_file_name()

	results = bq_client.query(query).to_dataframe()

	results.to_csv(file_name, sep='|', encoding='utf-8', index=False, header=True)

query_data()

In [172]:
def load_bq():
	file_name = gen_file_name()

	bucket = bucket_client.get_bucket('bucket_drive_test')

	bucket.blob(file_name).upload_from_filename(file_name)

load_bq()



<details>
<small>

<summary>how ```response``` looks like</summary>

```py
response = {
	'files': [{'id': '1GETX0fkc1M5DL1XjzGwZ9iXr6Lm9GYdn', 'name': 'results_2025-03-10.csv'},
			{'id': '2HFTX0fkc1M5DL1XjzGwZ9iXr6Lm9GYdn', 'name': 'results_2025-03-10.csv'}
	]
}
```

</small>
</details>

In [173]:
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT = 'test_bucket_gdrive.json'
PARENT_FOLDER_ID = '1OLll_SpLxMhVC5kY3ISu0lf0cdF01_pP'

def authenticate():
	creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT, scopes=SCOPES)
	return creds

def load_gdrive():
	creds = authenticate()
	service = build('drive', 'v3', credentials=creds)
	file_name = gen_file_name()

	query = f"'{PARENT_FOLDER_ID}' in parents and trashed=false"

	# returns a list of file ids
	# e.g. 
	response = service.files().list(q=query, fields='files(id, name)').execute()

	# .get() retrieves the value for a given key. 
	# if the key is missing, returns default value (or None) instead of raising KeyError.
	files = response.get('files') # response.get('files', [])

	if files:
		for file in files:
			service.files().delete(fileId=file['id']).execute()

	file_metadata = {
		'name': file_name,
		# parent must be a LIST
		'parents': [PARENT_FOLDER_ID]
	}

	file = service.files().create(
		body=file_metadata,
		media_body=file_name # file path
	).execute()

load_gdrive()

In [174]:
os.remove(gen_file_name())